In [17]:
import torch
import numpy as np
import os
from glob import glob
import cv2
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision.models as models

In [18]:
use_cuda = torch.cuda.is_available()

if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [19]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [20]:
body_files = np.array(glob("body/*/*/*"))
#woman_files = np.array(glob("body/woman/*/*/*"))


print('There are %d total man images.' % len(body_files))
#print('There are %d total w0man images.' % len(woman_files))

There are 1202 total man images.


In [21]:
batch_size = 20
num_workers = 0

data_dir = '/Users/bjoh1/python/capst/body/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

In [22]:

standard_normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])

In [23]:
data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'val': transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'test': transforms.Compose([transforms.Resize(size=(224,224)),
                                     transforms.ToTensor(), 
                                     standard_normalization])
                  }

In [24]:

train_data = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms['val'])
test_data = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

In [25]:
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
loaders_scratch = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}

In [26]:
num_classes =6

In [27]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
       
        self.conv1 = nn.Conv2d(3, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

       
        self.pool = nn.MaxPool2d(2, 2)
        
     
        self.fc1 = nn.Linear(7*7*128, 500)
        self.fc2 = nn.Linear(500, num_classes) 
        
      
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
       
        x = x.view(-1, 7*7*128)
        
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        
        x = self.dropout(x)
        x = self.fc2(x)
        return x




model_scratch = Net()
print(model_scratch)


if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6272, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=6, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [28]:
criterion_scratch = nn.CrossEntropyLoss()


optimizer_scratch = optim.SGD(model_scratch.parameters(), lr = 0.1)

In [70]:

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, last_validation_loss=None):
  
    if last_validation_loss is not None:
        valid_loss_min = last_validation_loss
    else:
        valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        
        train_loss = 0.0
        valid_loss = 0.0
        
      
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            

            
            optimizer.zero_grad()
            
            output = model(data)
            
            
            loss = criterion(output, target)
            
            
            loss.backward()
            
          
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
            
       
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            
       
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
   
    return model

In [71]:
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'saved_models/model_scratch.pt')

Epoch 1, Batch 1 loss: 1.481710
Epoch: 1 	Training Loss: 1.521901 	Validation Loss: 1.692672
Validation loss decreased (inf --> 1.692672).  Saving model ...
Epoch 2, Batch 1 loss: 1.352634
Epoch: 2 	Training Loss: 1.507479 	Validation Loss: 1.672729
Validation loss decreased (1.692672 --> 1.672729).  Saving model ...
Epoch 3, Batch 1 loss: 1.443165
Epoch: 3 	Training Loss: 1.502134 	Validation Loss: 1.625467
Validation loss decreased (1.672729 --> 1.625467).  Saving model ...
Epoch 4, Batch 1 loss: 1.595167
Epoch: 4 	Training Loss: 1.479955 	Validation Loss: 1.699464
Epoch 5, Batch 1 loss: 1.526594
Epoch: 5 	Training Loss: 1.484836 	Validation Loss: 1.664969
Epoch 6, Batch 1 loss: 1.499678
Epoch: 6 	Training Loss: 1.468639 	Validation Loss: 1.684355
Epoch 7, Batch 1 loss: 1.535448
Epoch: 7 	Training Loss: 1.472836 	Validation Loss: 1.544594
Validation loss decreased (1.625467 --> 1.544594).  Saving model ...
Epoch 8, Batch 1 loss: 1.596037
Epoch: 8 	Training Loss: 1.466316 	Validation 

In [29]:
model_scratch.load_state_dict(torch.load('saved_models/model_scratch.pt'))

<All keys matched successfully>

In [30]:

def test(loaders, model, criterion, use_cuda):

    
    test_loss = 0.
    correct = 0.
    total = 0.

    for batch_idx, (data, target) in enumerate(loaders['test']):
        
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
      
        loss = criterion(output, target)
         
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
       
        pred = output.data.max(1, keepdim=True)[1]
        
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

   
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Test Loss: 1.719238


Test Accuracy: 23% (104/445)


In [31]:
loaders_transfer = loaders_scratch.copy()

In [32]:
model_transfer = models.resnet50(pretrained=True)

In [41]:

for param in model_transfer.parameters():
    param.requires_grad = False
model_transfer.fc = nn.Linear(2048, 500, bias=True)
model_transfer.fc1=nn.Linear(500,6,bias=True)
fc_parameters = model_transfer.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True

In [42]:
model_transfer

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [43]:
if use_cuda:
    model_transfer = model_transfer.cuda()
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.fc.parameters(), lr=0.1)

In [44]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        
        train_loss = 0.0
        valid_loss = 0.0
        
       
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            
            optimizer.zero_grad()
            
            output = model(data)
            
          
            loss = criterion(output, target)
            
            
            loss.backward()
            
           
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
        
       
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
           
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            
         
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
   
    return model

In [45]:
train(20, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'saved_models/model_transfer.pt')

Epoch 1, Batch 1 loss: 5.907687
Epoch: 1 	Training Loss: 13.889280 	Validation Loss: 15.557920
Validation loss decreased (inf --> 15.557920).  Saving model ...
Epoch 2, Batch 1 loss: 14.410159
Epoch: 2 	Training Loss: 6.976110 	Validation Loss: 4.348722
Validation loss decreased (15.557920 --> 4.348722).  Saving model ...
Epoch 3, Batch 1 loss: 3.243552
Epoch: 3 	Training Loss: 4.954136 	Validation Loss: 1.709683
Validation loss decreased (4.348722 --> 1.709683).  Saving model ...
Epoch 4, Batch 1 loss: 0.493733
Epoch: 4 	Training Loss: 3.815196 	Validation Loss: 2.845033
Epoch 5, Batch 1 loss: 5.181374
Epoch: 5 	Training Loss: 4.425727 	Validation Loss: 2.266679
Epoch 6, Batch 1 loss: 1.897459
Epoch: 6 	Training Loss: 3.159636 	Validation Loss: 5.452839
Epoch 7, Batch 1 loss: 6.369069
Epoch: 7 	Training Loss: 3.019799 	Validation Loss: 3.484271
Epoch 8, Batch 1 loss: 2.874344
Epoch: 8 	Training Loss: 2.698442 	Validation Loss: 1.838931
Epoch 9, Batch 1 loss: 2.029668
Epoch: 9 	Trainin

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
model_transfer.load_state_dict(torch.load('saved_models/model_transfer.pt'))

<All keys matched successfully>

In [47]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 8.506269


Test Accuracy: 26% (118/445)


In [86]:
class_names = [item[4:].replace("_", " ") for item in loaders_transfer['train'].dataset.classes]

In [87]:
loaders_transfer['train'].dataset.classes[:10]

['1.bigman',
 '1.bigwoman',
 '2.heavy_man',
 '2.heavy_woman',
 '3.norman',
 '3.norwoman']

In [88]:

class_names[:10]

['gman', 'gwoman', 'avy man', 'avy woman', 'rman', 'rwoman']

In [89]:
def load_input_image(img_path):    
    image = Image.open(img_path).convert('RGB')
    prediction_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                     transforms.ToTensor(), 
                                     standard_normalization])

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = prediction_transform(image)[:3,:,:].unsqueeze(0)
    return image

In [90]:

def body(model, class_names, img_path):
    # load the image and return the predicted breed
    img = load_input_image(img_path)
    model = model.cpu()
    model.eval()
    idx = torch.argmax(model(img))
    return class_names[idx]

In [103]:
os.getcwd()

'C:\\Users\\bjoh1\\python\\capst'

In [121]:
for img_file in os.listdir('./images'):
    img_path = os.path.join('./images', img_file)
    predition = body(model_transfer, class_names, img_path)
    print("image_file_name: {0}, \t predition breed: {1}".format(img_path, predition))

image_file_name: ./images\하체비만50.jpg, 	 predition breed: rwoman
